In [1]:
# Macのみ
#!pip install selenium
#!brew install cask
#!brew cask install chromedriver

In [1]:
# スクレイプに使う
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions

# 検索ワードのURLエンコードに使う
import urllib.parse

# データフレーム
import pandas as pd
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 200)

# とりあえず、使ってないかも
import numpy as np

# スクレイピングの時間間隔をあけるために使う
from time import sleep

# 日時の取得に使う
import datetime

# excel出力に使う
import openpyxl

import re

In [2]:
# parameters


In [3]:
# ブラウザを開く
# chrome以外も使えるらしい
options = ChromeOptions()
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)


In [4]:
# 開催日毎のURLを取得する

race_date_col = ['race_date', 'race_date_url']
race_date_df = pd.DataFrame(columns = race_date_col, dtype = 'str')
race_date_df.describe

# for year in range(2009, 2019):
for year in range(2018, 2019):
#     for month in range(1, 13):
    for month in range(12, 13):
        url = 'http://www2.keiba.go.jp/KeibaWeb/MonthlyConveneInfo/MonthlyConveneInfoTop?k_year=' + str(year) + '&k_month=' + str(month)
#         print(url)
        # 検索結果ページへアクセス
        browser.get(url)
        xp_oi = '//*[@id="mainContainer"]/article/div/div[2]/table/tbody/tr[10]/td/a'
        elms_oi = browser.find_elements_by_xpath(xp_oi)
        for index, elm_oi in enumerate(elms_oi):
            race_date_url = elm_oi.get_attribute('href')
            race_date = race_date_url.split('=')[1].replace('%2f', '').split('&')[0]
            print(race_date_url)
            print(race_date)
            s = pd.Series([race_date, race_date_url], index = ['race_date', 'race_date_url'])
            race_date_df = race_date_df.append(s, ignore_index = True)
        sleep(1)

race_date_df


http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f03&k_babaCode=20
20181203
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f04&k_babaCode=20
20181204
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f05&k_babaCode=20
20181205
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f06&k_babaCode=20
20181206
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f07&k_babaCode=20
20181207
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f25&k_babaCode=20
20181225
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f26&k_babaCode=20
20181226
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f27&k_babaCode=20
20181227
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f29&k_babaCode=20
20181229
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList

,race_date,race_date_url
0,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
1,20181204,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
2,20181205,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
3,20181206,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
4,20181207,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
5,20181225,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
6,20181226,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
7,20181227,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
8,20181229,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
9,20181230,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...


In [5]:
# レース毎のURLを取得する

race_col = [
    'kyousou',
    'hassoujikoku',
    'henkou',
    'kyousoushurui',
    'kyousoumei',
    'course',
    'tenkou',
    'baba',
    'tousuu',
    'race_url',
    'result_url'
]


race_df = pd.DataFrame(columns = race_date_col + race_col, dtype = 'str')
race_df.describe

for key, row in race_date_df.iterrows():
    print(key)
    print(row['race_date_url'])
    race_date = row['race_date']
    race_date_url = row['race_date_url']
    browser.get(race_date_url)
    xp_race = '//*[@id="mainContainer"]/article[1]/div/section/table[1]/tbody/tr'
    elms_tr = browser.find_elements_by_xpath(xp_race)
    for index, elm_tr in enumerate(elms_tr):
        if index < 2:
            continue
        print('elm_tr: ' + str(index) + '===' + elm_tr.text)
        elms_td = elm_tr.find_elements_by_tag_name('td')
        
        kyousou = elms_td[0].text
        hassoujikoku = elms_td[1].text
        henkou = elms_td[2].text
        kyousoushurui = elms_td[3].text
        kyousoumei = elms_td[4].text
        course = elms_td[5].text
        tenkou = elms_td[6].text
        baba = elms_td[7].text
        tousuu = elms_td[8].text
        race_url = elms_td[4].find_element_by_tag_name('a').get_attribute('href')
        print(race_url)
        result_url = elms_td[9].find_elements_by_tag_name('a')[-1].get_attribute('href')
#         s = pd.Series([race_date, race_date_url, kyousou,hassoujikoku,henkou,kyousoushurui,kyousoumei,course,tenkou,baba,tousuu, race_url, result_url], index = race_col)
        s_ = pd.Series([kyousou,hassoujikoku,henkou,kyousoushurui,kyousoumei,course,tenkou,baba,tousuu, race_url, result_url],
                       index = ['kyousou', 'hassoujikoku', 'henkou', 'kyousoushurui', 'kyousoumei', 'course', 'tenkou', 'baba', 'tousuu', 'race_url', 'result_url'])
        s = pd.concat([row, s_])
        race_df = race_df.append(s, ignore_index = True)
    # アクセスの時間間隔をあける。絶対に入れること
    sleep(1)
#     break

race_df


0
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/RaceList?k_raceDate=2018%2f12%2f03&k_babaCode=20
elm_tr: 2===1R 14:50 ２歳７０万円以下 右1600m 曇 良 8 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f03&k_raceNo=1&k_babaCode=20
elm_tr: 3===2R 15:20 ２歳７０万円以下 右1400m 曇 良 11 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f03&k_raceNo=2&k_babaCode=20
elm_tr: 4===3R 15:50 ２歳７０万円以下 右1200m 曇 良 14 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f03&k_raceNo=3&k_babaCode=20
elm_tr: 5===4R 16:20 ３歳５０万円以下 右1400m 曇 良 12 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f03&k_raceNo=4&k_babaCode=20
elm_tr: 6===5R 16:55 Ｃ２三 四 右1600m 曇 良 14 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f03&k_raceNo=5&k_babaCode=20
elm_tr: 7===6R 17:25 Ｃ２三 四 右1200m 曇 良 15 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2

elm_tr: 3===2R 15:20 品川区内のゆるきゃら大集合！賞３歳２００ 右1200m 曇 不良 14 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f07&k_raceNo=2&k_babaCode=20
elm_tr: 4===3R 15:50 特別 Ｊ指 ひばり特別２歳選抜 右1600m 曇 不良 12 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f07&k_raceNo=3&k_babaCode=20
elm_tr: 5===4R 16:20 日曜開庁利用者３０万人突破賞２歳選抜特選 右1200m 曇 不良 8 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f07&k_raceNo=4&k_babaCode=20
elm_tr: 6===5R 16:55 品川区と高知県との連携協定Ｃ２十二 十三 右1600m 曇 不良 16 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f07&k_raceNo=5&k_babaCode=20
elm_tr: 7===6R 17:25 俺たちはシナスイ族賞Ｃ２十二 十三 右1200m 曇 不良 13 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f07&k_raceNo=6&k_babaCode=20
elm_tr: 8===7R 17:55 人権尊重都市品川宣言２５周年賞Ｃ１Ｃ２選 右1600m 曇 不良 9 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f07&k_raceNo=

elm_tr: 4===3R 11:56 ラーメンフェス２０１８＠ＴＣＫＣ２七 八 右1600m 晴 良 16 オッズ 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f29&k_raceNo=3&k_babaCode=20
elm_tr: 5===4R 12:26 有 的中祈願隊賞Ｃ２七 八 右1200m 晴 良 16 オッズ 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f29&k_raceNo=4&k_babaCode=20
elm_tr: 6===5R 13:00 有 藤田ニコルさんがＴＣＫに来場賞Ｃ２五 六 右1600m 晴 良 16 オッズ 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f29&k_raceNo=5&k_babaCode=20
elm_tr: 7===6R 13:35 有 川栄李奈さんがＴＣＫに来場賞Ｃ２五 六 右1200m 晴 良 16 オッズ 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f29&k_raceNo=6&k_babaCode=20
elm_tr: 8===7R 14:15 有 吉谷彩子さんがＴＣＫに来場賞Ｃ１八 九 十 右1600m 晴 良 14 オッズ 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f29&k_raceNo=7&k_babaCode=20
elm_tr: 9===8R 15:00 有 ＴＣＫイメージキャラクター来Ｃ１八 九 十 右1400m 晴 良 13 オッズ 映像 成績
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_rac

,race_date,race_date_url,kyousou,hassoujikoku,henkou,kyousoushurui,kyousoumei,course,tenkou,baba,tousuu,race_url,result_url
0,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,1R,14:50,,,２歳７０万円以下,右1600m,曇,良,8,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
1,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,2R,15:20,,,２歳７０万円以下,右1400m,曇,良,11,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
2,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,3R,15:50,,,２歳７０万円以下,右1200m,曇,良,14,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
3,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,4R,16:20,,,３歳５０万円以下,右1400m,曇,良,12,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
4,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,5R,16:55,,,Ｃ２三 四,右1600m,曇,良,14,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
5,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,6R,17:25,,,Ｃ２三 四,右1200m,曇,良,15,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
6,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,7R,17:55,有,,Ｃ２一 二,右1600m,曇,良,15,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
7,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,8R,18:33,,,Ｃ２一 二,右1200m,曇,良,13,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
8,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,9R,19:03,,,Ｃ１一 二,右1600m,曇,良,13,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...
9,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,10R,19:35,,,ローズマリー賞Ｃ１一 二,右1200m,曇,良,12,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...


In [6]:
horse_col = [
    'wakuban',
    'umaban',
    'bamei',
    'seirei',
    'keiro',
    'chichimei',
    'hahamei',
    'hahachichimei',
    'kishu',
    'futanjuuryou',
    'kijouseiseki',
    'choukyoushi',
    'banushi',
    'seisanbokujou',
    'odds',
    'ninki',
    'bataijuu',
    'zougen',
    'henkoujouhou',
    'r1_chakujun',
    'r1_kyousoubi',
    'r1_babajoutai',
    'r1_shussoutousuu',
    'r1_keibajou',
    'r1_course',
    'r1_umaban',
    'r1_racemei',
    'r1_ninki',
    'r1_bataijuu',
    'r1_genryou',
    'r1_kishu',
    'r1_futanjuuryou',
    'r1_time',
    'r1_cornertsuukajun',
    'r1_agari3ftime',
    'r1_1chakubatonotimesa',
    'r1_1chakubamei',
    'r2_chakujun',
    'r2_kyousoubi',
    'r2_babajoutai',
    'r2_shussoutousuu',
    'r2_keibajou',
    'r2_course',
    'r2_umaban',
    'r2_racemei',
    'r2_ninki',
    'r2_bataijuu',
    'r2_genryou',
    'r2_kishu',
    'r2_futanjuuryou',
    'r2_time',
    'r2_cornertsuukajun',
    'r2_agari3ftime',
    'r2_1chakubatonotimesa',
    'r2_1chakubamei',
    'r3_chakujun',
    'r3_kyousoubi',
    'r3_babajoutai',
    'r3_shussoutousuu',
    'r3_keibajou',
    'r3_course',
    'r3_umaban',
    'r3_racemei',
    'r3_ninki',
    'r3_bataijuu',
    'r3_genryou',
    'r3_kishu',
    'r3_futanjuuryou',
    'r3_time',
    'r3_cornertsuukajun',
    'r3_agari3ftime',
    'r3_1chakubatonotimesa',
    'r3_1chakubamei',
    'r4_chakujun',
    'r4_kyousoubi',
    'r4_babajoutai',
    'r4_shussoutousuu',
    'r4_keibajou',
    'r4_course',
    'r4_umaban',
    'r4_racemei',
    'r4_ninki',
    'r4_bataijuu',
    'r4_genryou',
    'r4_kishu',
    'r4_futanjuuryou',
    'r4_time',
    'r4_cornertsuukajun',
    'r4_agari3ftime',
    'r4_1chakubatonotimesa',
    'r4_1chakubamei',
    'r5_chakujun',
    'r5_kyousoubi',
    'r5_babajoutai',
    'r5_shussoutousuu',
    'r5_keibajou',
    'r5_course',
    'r5_umaban',
    'r5_racemei',
    'r5_ninki',
    'r5_bataijuu',
    'r5_genryou',
    'r5_kishu',
    'r5_futanjuuryou',
    'r5_time',
    'r5_cornertsuukajun',
    'r5_agari3ftime',
    'r5_1chakubatonotimesa',
    'r5_1chakubamei',
    'zenseiseki',
    'dirthidarimawari',
    'dirtmigimawari',
    'toucourse'
]

# データフレームの初期化
horse_df = pd.DataFrame(columns = race_date_col + race_col + horse_col, dtype = 'str')
# horse_df.describe

for key, row in race_df.iterrows():
    print(key)
    print(row['race_url'])
    
#     if key < 68:
#         continue
    
    race_url = row['race_url']
    browser.get(race_url)
    
    xp_table = '//*[@id="mainContainer"]/article[1]/div/section[2]/table/tbody/tr'
    elms_tr = browser.find_elements_by_xpath(xp_table)

    for index, elm_tr in enumerate(elms_tr):
        if index < 2:
            continue
        print('elm_tr: ' + str(index) + ' / ' + str((index - 2) / 5))
        if index % 5 == 2:
            elms_td = elm_tr.find_elements_by_tag_name('td')

            i = 0
            class_name = elms_td[i].get_attribute('class')
            if class_name == "horseNum":
                wakuban = np.nan
            else:
                wakuban = elms_td[i].text.split()[0]
                i += 1
            umaban = elms_td[i].text
            i += 1
            bamei = elms_td[i].text
            i += 1
            kishu = elms_td[i].text
            i += 1
            if len(elms_td[i].text) > 0:
                odds = elms_td[i].text.split()[0]
                ninki = elms_td[i].text.split()[1]
            else:
                odds = np.nan
                ninki = np.nan
            
            i += 1
            if (len(elms_td[i].text) > 0 and elms_td[i].text.split()[0] != u'取止'):
                r1_chakujun = elms_td[i].text.split()[0]
                r1_kyousoubi = elms_td[i].text.split()[1]
                r1_babajoutai = elms_td[i].text.split()[2]
                r1_shussoutousuu = elms_td[i].text.split()[3]
                r1_keibajou = elms_td[i].text.split()[4]
                r1_course = elms_td[i].text.split()[5]
                r1_umaban = elms_td[i].text.split()[6]
            else:
                r1_chakujun = np.nan
                r1_kyousoubi = np.nan
                r1_babajoutai = np.nan
                r1_shussoutousuu = np.nan
                r1_keibajou = np.nan
                r1_course = np.nan
                r1_umaban = np.nan
            i += 1
            if (len(elms_td[i].text) > 0 and elms_td[i].text.split()[0] != u'取止'):
                r2_chakujun = elms_td[i].text.split()[0]
                r2_kyousoubi = elms_td[i].text.split()[1]
                r2_babajoutai = elms_td[i].text.split()[2]
                r2_shussoutousuu = elms_td[i].text.split()[3]
                r2_keibajou = elms_td[i].text.split()[4]
                r2_course = elms_td[i].text.split()[5]
                r2_umaban = elms_td[i].text.split()[6]
            else:
                r2_chakujun = np.nan
                r2_kyousoubi = np.nan
                r2_babajoutai = np.nan
                r2_shussoutousuu = np.nan
                r2_keibajou = np.nan
                r2_course = np.nan
                r2_umaban = np.nan
            i += 1
            if (len(elms_td[i].text) > 0 and elms_td[i].text.split()[0] != u'取止'):
                r3_chakujun = elms_td[i].text.split()[0]
                r3_kyousoubi = elms_td[i].text.split()[1]
                r3_babajoutai = elms_td[i].text.split()[2]
                r3_shussoutousuu = elms_td[i].text.split()[3]
                r3_keibajou = elms_td[i].text.split()[4]
                r3_course = elms_td[i].text.split()[5]
                r3_umaban = elms_td[i].text.split()[6]
            else:
                r3_chakujun = np.nan
                r3_kyousoubi = np.nan
                r3_babajoutai = np.nan
                r3_shussoutousuu = np.nan
                r3_keibajou = np.nan
                r3_course = np.nan
                r3_umaban = np.nan
            i += 1
            if (len(elms_td[i].text) > 0 and elms_td[i].text.split()[0] != u'取止'):
                r4_chakujun = elms_td[i].text.split()[0]
                r4_kyousoubi = elms_td[i].text.split()[1]
                r4_babajoutai = elms_td[i].text.split()[2]
                r4_shussoutousuu = elms_td[i].text.split()[3]
                r4_keibajou = elms_td[i].text.split()[4]
                r4_course = elms_td[i].text.split()[5]
                r4_umaban = elms_td[i].text.split()[6]
            else:
                r4_chakujun = np.nan
                r4_kyousoubi = np.nan
                r4_babajoutai = np.nan
                r4_shussoutousuu = np.nan
                r4_keibajou = np.nan
                r4_course = np.nan
                r4_umaban = np.nan
            i += 1
            if (len(elms_td[i].text) > 0 and elms_td[i].text.split()[0] != u'取止'):
                r5_chakujun = elms_td[i].text.split()[0]
                r5_kyousoubi = elms_td[i].text.split()[1]
                r5_babajoutai = elms_td[i].text.split()[2]
                r5_shussoutousuu = elms_td[i].text.split()[3]
                r5_keibajou = elms_td[i].text.split()[4]
                r5_course = elms_td[i].text.split()[5]
                r5_umaban = elms_td[i].text.split()[6]
            else:
                r5_chakujun = np.nan
                r5_kyousoubi = np.nan
                r5_babajoutai = np.nan
                r5_shussoutousuu = np.nan
                r5_keibajou = np.nan
                r5_course = np.nan
                r5_umaban = np.nan
            i += 1
            zenseiseki = elms_td[i].text.split()[0]
            dirthidarimawari = elms_td[i].text.split()[1]
            dirtmigimawari = elms_td[i].text.split()[2]
            toucourse = elms_td[i].text.split()[3]

        if index % 5 == 3:
            elms_td = elm_tr.find_elements_by_tag_name('td')

            seirei = elms_td[0].text
            keiro = elms_td[1].text
            futanjuuryou = elms_td[2].text.split()[0]
            kijouseiseki = elms_td[2].text.split()[1]
            i = 3
            if len(elms_td[i].text) > 0:
                r1_racemei = elms_td[i].text
            else:
                r1_racemei = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r2_racemei = elms_td[i].text
            else:
                r2_racemei = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r3_racemei = elms_td[i].text
            else:
                r3_racemei = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r4_racemei = elms_td[i].text
            else:
                r4_racemei = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r5_racemei = elms_td[i].text
            else:
                r5_racemei = np.nan

        if index % 5 == 4:
            elms_td = elm_tr.find_elements_by_tag_name('td')

            chichimei = elms_td[0].text
            choukyoushi = elms_td[1].text
            bataijuu = elms_td[2].text.split()[0]
            if len(elms_td[2].text.split()) > 1:
                zougen = elms_td[2].text.split()[1]
            else:
                zougen = np.nan
            
            i = 3
            if len(elms_td[i].text) > 0:
                r1_ninki = elms_td[i].text.split()[0]
                r1_bataijuu = elms_td[i].text.split()[1]
                r1_kishu = elms_td[i].text.split()[-2]
                r1_futanjuuryou = elms_td[i].text.split()[-1]
            else:
                r1_ninki = np.nan
                r1_bataijuu = np.nan
                r1_kishu = np.nan
                r1_futanjuuryou = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r2_ninki = elms_td[i].text.split()[0]
                r2_bataijuu = elms_td[i].text.split()[1]
                r2_kishu = elms_td[i].text.split()[-2]
                r2_futanjuuryou = elms_td[i].text.split()[-1]
            else:
                r2_ninki = np.nan
                r2_bataijuu = np.nan
                r2_kishu = np.nan
                r2_futanjuuryou = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r3_ninki = elms_td[i].text.split()[0]
                r3_bataijuu = elms_td[i].text.split()[1]
                r3_kishu = elms_td[i].text.split()[-2]
                r3_futanjuuryou = elms_td[i].text.split()[-1]
            else:
                r3_ninki = np.nan
                r3_bataijuu = np.nan
                r3_kishu = np.nan
                r3_futanjuuryou = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r4_ninki = elms_td[i].text.split()[0]
                r4_bataijuu = elms_td[i].text.split()[1]
                r4_kishu = elms_td[i].text.split()[-2]
                r4_futanjuuryou = elms_td[i].text.split()[-1]
            else:
                r4_ninki = np.nan
                r4_bataijuu = np.nan
                r4_kishu = np.nan
                r4_futanjuuryou = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r5_ninki = elms_td[i].text.split()[0]
                r5_bataijuu = elms_td[i].text.split()[1]
                r5_kishu = elms_td[i].text.split()[-2]
                r5_futanjuuryou = elms_td[i].text.split()[-1]
            else:
                r5_ninki = np.nan
                r5_bataijuu = np.nan
                r5_kishu = np.nan
                r5_futanjuuryou = np.nan

        if index % 5 == 0:
            elms_td = elm_tr.find_elements_by_tag_name('td')

            hahamei = elms_td[0].text
            banushi = elms_td[1].text
            i = 2
            if len(elms_td[i].text) > 0:
                r1_time = elms_td[i].text.split()[0]
                if len(elms_td[i].text.split()) > 2:
                    r1_cornertsuukajun = elms_td[i].text.split()[1]
                else:
                    r1_cornertsuukajun = np.nan
                r1_agari3ftime = elms_td[i].text.split()[-1]
            else:
                r1_time = np.nan
                r1_cornertsuukajun = np.nan
                r1_agari3ftime = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r2_time = elms_td[i].text.split()[0]
                if len(elms_td[i].text.split()) > 2:
                    r2_cornertsuukajun = elms_td[i].text.split()[1]
                else:
                    r2_cornertsuukajun = np.nan
                r2_agari3ftime = elms_td[i].text.split()[-1]
            else:
                r2_time = np.nan
                r2_cornertsuukajun = np.nan
                r2_agari3ftime = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r3_time = elms_td[i].text.split()[0]
                if len(elms_td[i].text.split()) > 2:
                    r3_cornertsuukajun = elms_td[i].text.split()[1]
                else:
                    r3_cornertsuukajun = np.nan
                r3_agari3ftime = elms_td[i].text.split()[-1]
            else:
                r3_time = np.nan
                r3_cornertsuukajun = np.nan
                r3_agari3ftime = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r4_time = elms_td[i].text.split()[0]
                if len(elms_td[i].text.split()) > 2:
                    r4_cornertsuukajun = elms_td[i].text.split()[1]
                else:
                    r4_cornertsuukajun = np.nan
                r4_agari3ftime = elms_td[i].text.split()[-1]
            else:
                r4_time = np.nan
                r4_cornertsuukajun = np.nan
                r4_agari3ftime = np.nan
            i += 1
            if len(elms_td[i].text) > 0:
                r5_time = elms_td[i].text.split()[0]
                if len(elms_td[i].text.split()) > 2:
                    r5_cornertsuukajun = elms_td[i].text.split()[1]
                else:
                    r5_cornertsuukajun = np.nan
                r5_agari3ftime = elms_td[i].text.split()[-1]
            else:
                r5_time = np.nan
                r5_cornertsuukajun = np.nan
                r5_agari3ftime = np.nan

        if index % 5 == 1:
            elms_td = elm_tr.find_elements_by_tag_name('td')

            hahachichimei = elms_td[0].text
            seisanbokujou = elms_td[1].text
            if len(elms_td[2].text) > 0:
                henkoujouhou = elms_td[2].text
            else:
                henkoujouhou = np.nan
            i = 3
            if len(elms_td[i].text.split()) > 1:
                r1_1chakubatonotimesa = elms_td[i].text.split()[0]
                r1_1chakubamei = elms_td[i].text.split()[1]
            else:
                r1_1chakubatonotimesa = np.nan
                r1_1chakubamei = np.nan
            i += 1
            if len(elms_td[i].text.split()) > 1:
                r2_1chakubatonotimesa = elms_td[i].text.split()[0]
                r2_1chakubamei = elms_td[i].text.split()[1]
            else:
                r2_1chakubatonotimesa = np.nan
                r2_1chakubamei = np.nan
            i += 1
            if len(elms_td[i].text.split()) > 1:
                r3_1chakubatonotimesa = elms_td[i].text.split()[0]
                r3_1chakubamei = elms_td[i].text.split()[1]
            else:
                r3_1chakubatonotimesa = np.nan
                r3_1chakubamei = np.nan
            i += 1
            if len(elms_td[i].text.split()) > 1:
                r4_1chakubatonotimesa = elms_td[i].text.split()[0]
                r4_1chakubamei = elms_td[i].text.split()[1]
            else:
                r4_1chakubatonotimesa = np.nan
                r4_1chakubamei = np.nan
            i += 1
            if len(elms_td[i].text.split()) > 1:
                r5_1chakubatonotimesa = elms_td[i].text.split()[0]
                r5_1chakubamei = elms_td[i].text.split()[1]
            else:
                r5_1chakubatonotimesa = np.nan
                r5_1chakubamei = np.nan
        
        if index % 5 == 1:
            s_ = pd.Series([wakuban,umaban,bamei,kishu,odds,ninki,r1_chakujun,r1_kyousoubi,r1_babajoutai,r1_shussoutousuu,r1_keibajou,r1_course,r1_umaban,r2_chakujun,r2_kyousoubi,r2_babajoutai,r2_shussoutousuu,r2_keibajou,r2_course,r2_umaban,r3_chakujun,r3_kyousoubi,r3_babajoutai,r3_shussoutousuu,r3_keibajou,r3_course,r3_umaban,r4_chakujun,r4_kyousoubi,r4_babajoutai,r4_shussoutousuu,r4_keibajou,r4_course,r4_umaban,r5_chakujun,r5_kyousoubi,r5_babajoutai,r5_shussoutousuu,r5_keibajou,r5_course,r5_umaban,zenseiseki,dirthidarimawari,dirtmigimawari,toucourse,seirei,keiro,futanjuuryou,kijouseiseki,r1_racemei,r2_racemei,r3_racemei,r4_racemei,r5_racemei,chichimei,choukyoushi,bataijuu,zougen,r1_ninki,r1_bataijuu,r1_kishu,r1_futanjuuryou,r2_ninki,r2_bataijuu,r2_kishu,r2_futanjuuryou,r3_ninki,r3_bataijuu,r3_kishu,r3_futanjuuryou,r4_ninki,r4_bataijuu,r4_kishu,r4_futanjuuryou,r5_ninki,r5_bataijuu,r5_kishu,r5_futanjuuryou,hahamei,banushi,r1_time,r1_cornertsuukajun,r1_agari3ftime,r2_time,r2_cornertsuukajun,r2_agari3ftime,r3_time,r3_cornertsuukajun,r3_agari3ftime,r4_time,r4_cornertsuukajun,r4_agari3ftime,r5_time,r5_cornertsuukajun,r5_agari3ftime,hahachichimei,seisanbokujou,henkoujouhou,r1_1chakubatonotimesa,r1_1chakubamei,r2_1chakubatonotimesa,r2_1chakubamei,r3_1chakubatonotimesa,r3_1chakubamei,r4_1chakubatonotimesa,r4_1chakubamei,r5_1chakubatonotimesa,r5_1chakubamei],
                index = ['wakuban','umaban','bamei','kishu','odds','ninki','r1_chakujun','r1_kyousoubi','r1_babajoutai','r1_shussoutousuu','r1_keibajou','r1_course','r1_umaban','r2_chakujun','r2_kyousoubi','r2_babajoutai','r2_shussoutousuu','r2_keibajou','r2_course','r2_umaban','r3_chakujun','r3_kyousoubi','r3_babajoutai','r3_shussoutousuu','r3_keibajou','r3_course','r3_umaban','r4_chakujun','r4_kyousoubi','r4_babajoutai','r4_shussoutousuu','r4_keibajou','r4_course','r4_umaban','r5_chakujun','r5_kyousoubi','r5_babajoutai','r5_shussoutousuu','r5_keibajou','r5_course','r5_umaban','zenseiseki','dirthidarimawari','dirtmigimawari','toucourse','seirei','keiro','futanjuuryou','kijouseiseki','r1_racemei','r2_racemei','r3_racemei','r4_racemei','r5_racemei','chichimei','choukyoushi','bataijuu','zougen','r1_ninki','r1_bataijuu','r1_kishu','r1_futanjuuryou','r2_ninki','r2_bataijuu','r2_kishu','r2_futanjuuryou','r3_ninki','r3_bataijuu','r3_kishu','r3_futanjuuryou','r4_ninki','r4_bataijuu','r4_kishu','r4_futanjuuryou','r5_ninki','r5_bataijuu','r5_kishu','r5_futanjuuryou','hahamei','banushi','r1_time','r1_cornertsuukajun','r1_agari3ftime','r2_time','r2_cornertsuukajun','r2_agari3ftime','r3_time','r3_cornertsuukajun','r3_agari3ftime','r4_time','r4_cornertsuukajun','r4_agari3ftime','r5_time','r5_cornertsuukajun','r5_agari3ftime','hahachichimei','seisanbokujou','henkoujouhou','r1_1chakubatonotimesa','r1_1chakubamei','r2_1chakubatonotimesa','r2_1chakubamei','r3_1chakubatonotimesa','r3_1chakubamei','r4_1chakubatonotimesa','r4_1chakubamei','r5_1chakubatonotimesa','r5_1chakubamei'])
    #         s = pd.Series([kyousoubi, weekday, kyousou, hassoujikoku, henkou, kyousoushurui, kyousoumei, course, tenkou, baba, tousuu, wakuban, umaban, bamei, seirei, keiro, chichimei, hahamei, hahachichimei, kishu, futanjuuryou, kijouseiseki, choukyoushi, banushi, seisanbokujou, odds, ninki, bataijuu, zougen, henkoujouhou, r1_chakujun, r1_kyousoubi, r1_babajoutai, r1_shussoutousuu, r1_keibajou, r1_course, r1_umaban, r1_racemei, r1_ninki, r1_bataijuu, r1_genryou, r1_kishu, r1_futanjuuryou, r1_time, r1_cornertsuukajun, r1_agari3ftime, r1_1chakubatonotimesa, r1_1chakubamei, r2_chakujun, r2_kyousoubi, r2_babajoutai, r2_shussoutousuu, r2_keibajou, r2_course, r2_umaban, r2_racemei, r2_ninki, r2_bataijuu, r2_genryou, r2_kishu, r2_futanjuuryou, r2_time, r2_cornertsuukajun, r2_agari3ftime, r2_1chakubatonotimesa, r2_1chakubamei, r3_chakujun, r3_kyousoubi, r3_babajoutai, r3_shussoutousuu, r3_keibajou, r3_course, r3_umaban, r3_racemei, r3_ninki, r3_bataijuu, r3_genryou, r3_kishu, r3_futanjuuryou, r3_time, r3_cornertsuukajun, r3_agari3ftime, r3_1chakubatonotimesa, r3_1chakubamei, r4_chakujun, r4_kyousoubi, r4_babajoutai, r4_shussoutousuu, r4_keibajou, r4_course, r4_umaban, r4_racemei, r4_ninki, r4_bataijuu, r4_genryou, r4_kishu, r4_futanjuuryou, r4_time, r4_cornertsuukajun, r4_agari3ftime, r4_1chakubatonotimesa, r4_1chakubamei, r5_chakujun, r5_kyousoubi, r5_babajoutai, r5_shussoutousuu, r5_keibajou, r5_course, r5_umaban, r5_racemei, r5_ninki, r5_bataijuu, r5_genryou, r5_kishu, r5_futanjuuryou, r5_time, r5_cornertsuukajun, r5_agari3ftime, r5_1chakubatonotimesa, r5_1chakubamei, zenseiseki, dirthidarimawari, dirtmigimawari, toucourse],
    #             index = ['kyousoubi', 'weekday', 'kyousou', 'hassoujikoku', 'henkou', 'kyousoushurui', 'kyousoumei', 'course', 'tenkou', 'baba', 'tousuu', 'wakuban', 'umaban', 'bamei', 'seirei', 'keiro', 'chichimei', 'hahamei', 'hahachichimei', 'kishu', 'futanjuuryou', 'kijouseiseki', 'choukyoushi', 'banushi', 'seisanbokujou', 'odds', 'ninki', 'bataijuu', 'zougen', 'henkoujouhou', 'r1_chakujun', 'r1_kyousoubi', 'r1_babajoutai', 'r1_shussoutousuu', 'r1_keibajou', 'r1_course', 'r1_umaban', 'r1_racemei', 'r1_ninki', 'r1_bataijuu', 'r1_genryou', 'r1_kishu', 'r1_futanjuuryou', 'r1_time', 'r1_cornertsuukajun', 'r1_agari3ftime', 'r1_1chakubatonotimesa', 'r1_1chakubamei', 'r2_chakujun', 'r2_kyousoubi', 'r2_babajoutai', 'r2_shussoutousuu', 'r2_keibajou', 'r2_course', 'r2_umaban', 'r2_racemei', 'r2_ninki', 'r2_bataijuu', 'r2_genryou', 'r2_kishu', 'r2_futanjuuryou', 'r2_time', 'r2_cornertsuukajun', 'r2_agari3ftime', 'r2_1chakubatonotimesa', 'r2_1chakubamei', 'r3_chakujun', 'r3_kyousoubi', 'r3_babajoutai', 'r3_shussoutousuu', 'r3_keibajou', 'r3_course', 'r3_umaban', 'r3_racemei', 'r3_ninki', 'r3_bataijuu', 'r3_genryou', 'r3_kishu', 'r3_futanjuuryou', 'r3_time', 'r3_cornertsuukajun', 'r3_agari3ftime', 'r3_1chakubatonotimesa', 'r3_1chakubamei', 'r4_chakujun', 'r4_kyousoubi', 'r4_babajoutai', 'r4_shussoutousuu', 'r4_keibajou', 'r4_course', 'r4_umaban', 'r4_racemei', 'r4_ninki', 'r4_bataijuu', 'r4_genryou', 'r4_kishu', 'r4_futanjuuryou', 'r4_time', 'r4_cornertsuukajun', 'r4_agari3ftime', 'r4_1chakubatonotimesa', 'r4_1chakubamei', 'r5_chakujun', 'r5_kyousoubi', 'r5_babajoutai', 'r5_shussoutousuu', 'r5_keibajou', 'r5_course', 'r5_umaban', 'r5_racemei', 'r5_ninki', 'r5_bataijuu', 'r5_genryou', 'r5_kishu', 'r5_futanjuuryou', 'r5_time', 'r5_cornertsuukajun', 'r5_agari3ftime', 'r5_1chakubatonotimesa', 'r5_1chakubamei', 'zenseiseki', 'dirthidarimawari', 'dirtmigimawari', 'toucourse'])
            s = pd.concat([row, s_])
            horse_df = horse_df.append(s, ignore_index = True)
            
    sleep(1)
    break

horse_df


0
http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo/DebaTable?k_raceDate=2018%2f12%2f03&k_raceNo=1&k_babaCode=20
elm_tr: 2 / 0.0
elm_tr: 3 / 0.2
elm_tr: 4 / 0.4
elm_tr: 5 / 0.6
elm_tr: 6 / 0.8
elm_tr: 7 / 1.0
elm_tr: 8 / 1.2
elm_tr: 9 / 1.4
elm_tr: 10 / 1.6
elm_tr: 11 / 1.8
elm_tr: 12 / 2.0
elm_tr: 13 / 2.2
elm_tr: 14 / 2.4
elm_tr: 15 / 2.6
elm_tr: 16 / 2.8
elm_tr: 17 / 3.0
elm_tr: 18 / 3.2
elm_tr: 19 / 3.4
elm_tr: 20 / 3.6
elm_tr: 21 / 3.8
elm_tr: 22 / 4.0
elm_tr: 23 / 4.2
elm_tr: 24 / 4.4
elm_tr: 25 / 4.6
elm_tr: 26 / 4.8
elm_tr: 27 / 5.0
elm_tr: 28 / 5.2
elm_tr: 29 / 5.4
elm_tr: 30 / 5.6
elm_tr: 31 / 5.8
elm_tr: 32 / 6.0
elm_tr: 33 / 6.2
elm_tr: 34 / 6.4
elm_tr: 35 / 6.6
elm_tr: 36 / 6.8
elm_tr: 37 / 7.0
elm_tr: 38 / 7.2
elm_tr: 39 / 7.4
elm_tr: 40 / 7.6
elm_tr: 41 / 7.8


,race_date,race_date_url,kyousou,hassoujikoku,henkou,kyousoushurui,kyousoumei,course,tenkou,baba,tousuu,race_url,result_url,wakuban,umaban,bamei,seirei,keiro,chichimei,hahamei,hahachichimei,kishu,futanjuuryou,kijouseiseki,choukyoushi,banushi,seisanbokujou,odds,ninki,bataijuu,zougen,henkoujouhou,r1_chakujun,r1_kyousoubi,r1_babajoutai,r1_shussoutousuu,r1_keibajou,r1_course,r1_umaban,r1_racemei,r1_ninki,r1_bataijuu,r1_genryou,r1_kishu,r1_futanjuuryou,r1_time,r1_cornertsuukajun,r1_agari3ftime,r1_1chakubatonotimesa,r1_1chakubamei,r2_chakujun,r2_kyousoubi,r2_babajoutai,r2_shussoutousuu,r2_keibajou,r2_course,r2_umaban,r2_racemei,r2_ninki,r2_bataijuu,r2_genryou,r2_kishu,r2_futanjuuryou,r2_time,r2_cornertsuukajun,r2_agari3ftime,r2_1chakubatonotimesa,r2_1chakubamei,r3_chakujun,r3_kyousoubi,r3_babajoutai,r3_shussoutousuu,r3_keibajou,r3_course,r3_umaban,r3_racemei,r3_ninki,r3_bataijuu,r3_genryou,r3_kishu,r3_futanjuuryou,r3_time,r3_cornertsuukajun,r3_agari3ftime,r3_1chakubatonotimesa,r3_1chakubamei,r4_chakujun,r4_kyousoubi,r4_babajoutai,r4_shussoutousuu,r4_keibajou,r4_course,r4_umaban,r4_racemei,r4_ninki,r4_bataijuu,r4_genryou,r4_kishu,r4_futanjuuryou,r4_time,r4_cornertsuukajun,r4_agari3ftime,r4_1chakubatonotimesa,r4_1chakubamei,r5_chakujun,r5_kyousoubi,r5_babajoutai,r5_shussoutousuu,r5_keibajou,r5_course,r5_umaban,r5_racemei,r5_ninki,r5_bataijuu,r5_genryou,r5_kishu,r5_futanjuuryou,r5_time,r5_cornertsuukajun,r5_agari3ftime,r5_1chakubatonotimesa,r5_1chakubamei,zenseiseki,dirthidarimawari,dirtmigimawari,toucourse
0,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,1R,14:50,,,２歳７０万円以下,右1600m,曇,良,8,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,1,1,サンドムーン,牡2,栗毛,サウスヴィグラス,ジャングルムーン,（Ｓａｄｌｅｒ’ｓ Ｗｅｌｌｓ）,柏木健（大井）,55.0,0-0-0-0,市村誠（大井）,藤間淳一,阿部栄乃進,NaN,NaN,453,(+8),NaN,11,18.11.13,重,12頭,大井,右1200,6番,２歳９０万円以下,5人,445,NaN,的場文,55.0,1:19.9,11-11,41.3,3.6,ミコマイ,9,18.10.30,良,13頭,大井,右1200,2番,２歳５５万円以下,1人,437,NaN,的場文,55.0,1:18.5,10-10,40.4,2.1,タイムパワー,4,18.10.09,良,11頭,大井,右1200,8番,２歳４５万円以下,2人,442,NaN,真島大,55.0,1:18.1,8-8,40.7,1.6,マーブルリバース,6,18.09.21,不良,9頭,大井,右1200,4番,２歳新馬,6人,440,NaN,☆櫻井光,53.0,1:19.0,7-8,40.6,1.6,ミヤサンヴィッキー,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-0-0-4,0-0-0-0,0-0-0-4,0-0-0-4
1,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,1R,14:50,,,２歳７０万円以下,右1600m,曇,良,8,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,2,2,サブノスキヤネン,牝2,栗毛,ローエングリン,ミズサワスキヤネン,（スキャン）,笹川翼（大井）,54.0,0-0-0-1,鷹見浩（大井）,中川三郎,坂本健一,NaN,NaN,451,(-1),NaN,8,18.11.15,稍重,8頭,大井,右1600,5番,２歳新馬,4人,452,NaN,笹川翼,54.0,1:50.6,5-5-5-6,42.9,2.4,フェデルタ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-0-0-1,0-0-0-0,0-0-0-1,0-0-0-1
2,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,1R,14:50,,,２歳７０万円以下,右1600m,曇,良,8,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,3,3,エースティターン,牡2,鹿毛,ワークフォース,トリプルミッション,（ハービンジャー）,安藤洋（大井）,55.0,0-0-0-2,香取和（大井）,（有）オート・エー,社台ファーム,NaN,NaN,528,(-2),NaN,10,18.11.13,重,12頭,大井,右1400,2番,２歳９０万円以下,10人,530,NaN,安藤洋,55.0,1:34.2,12-10-10,42.2,3.8,キックザキャリバー,5,18.08.17,良,5頭,大井,右1400,4番,２歳新馬,5人,529,NaN,安藤洋,54.0,1:37.4,4-5-5,44.7,5.8,ハルサエウィズ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-0-0-2,0-0-0-0,0-0-0-2,0-0-0-2
3,20181203,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,1R,14:50,,,２歳７０万円以下,右1600m,曇,良,8,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,4,4,アルバストラーダ,牡2,青鹿毛,フリオーソ,チェリッシュレディ,（バブルガムフェロー）,矢野貴（大井）,55.0,0-0-0-0,月岡健（大井）,田村孝仁,ハシモトフアーム

In [8]:
horse_df

,race_date,race_date_url,kyousou,hassoujikoku,henkou,kyousoushurui,kyousoumei,course,tenkou,baba,tousuu,race_url,result_url,wakuban,umaban,bamei,seirei,keiro,chichimei,hahamei,hahachichimei,kishu,futanjuuryou,kijouseiseki,choukyoushi,banushi,seisanbokujou,odds,ninki,bataijuu,zougen,henkoujouhou,r1_chakujun,r1_kyousoubi,r1_babajoutai,r1_shussoutousuu,r1_keibajou,r1_course,r1_umaban,r1_racemei,r1_ninki,r1_bataijuu,r1_genryou,r1_kishu,r1_futanjuuryou,r1_time,r1_cornertsuukajun,r1_agari3ftime,r1_1chakubatonotimesa,r1_1chakubamei,r2_chakujun,r2_kyousoubi,r2_babajoutai,r2_shussoutousuu,r2_keibajou,r2_course,r2_umaban,r2_racemei,r2_ninki,r2_bataijuu,r2_genryou,r2_kishu,r2_futanjuuryou,r2_time,r2_cornertsuukajun,r2_agari3ftime,r2_1chakubatonotimesa,r2_1chakubamei,r3_chakujun,r3_kyousoubi,r3_babajoutai,r3_shussoutousuu,r3_keibajou,r3_course,r3_umaban,r3_racemei,r3_ninki,r3_bataijuu,r3_genryou,r3_kishu,r3_futanjuuryou,r3_time,r3_cornertsuukajun,r3_agari3ftime,r3_1chakubatonotimesa,r3_1chakubamei,r4_chakujun,r4_kyousoubi,r4_babajoutai,r4_shussoutousuu,r4_keibajou,r4_course,r4_umaban,r4_racemei,r4_ninki,r4_bataijuu,r4_genryou,r4_kishu,r4_futanjuuryou,r4_time,r4_cornertsuukajun,r4_agari3ftime,r4_1chakubatonotimesa,r4_1chakubamei,r5_chakujun,r5_kyousoubi,r5_babajoutai,r5_shussoutousuu,r5_keibajou,r5_course,r5_umaban,r5_racemei,r5_ninki,r5_bataijuu,r5_genryou,r5_kishu,r5_futanjuuryou,r5_time,r5_cornertsuukajun,r5_agari3ftime,r5_1chakubatonotimesa,r5_1chakubamei,zenseiseki,dirthidarimawari,dirtmigimawari,toucourse
0,20181225,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,9R,19:35,,,サンタクロース賞Ｃ１一 二 三,右1400m,晴,稍重,12,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,1,1,マントルパワー,牡3,栗毛,ローエングリン,メイショウブリーズ,（サンデーサイレンス）,笹川翼（大井）,56.0,1-1-0-1,橋本馬（大井）,和田博美,日西牧場,3.8,(2人気),492,(0),NaN,6,18.12.03,良,13頭,大井ナ,右1600,12番,Ｃ１一 二,3人,492,NaN,笹川翼,56.0,1:44.4,3-5-6-6,39.7,1.2,ミヤジフランガン,2,18.11.12,重,13頭,大井ナ,右1600,7番,Ｃ１一 二 三,5人,484,NaN,笹川翼,56.0,1:44.4,4-4-4-3,40.5,0.7,クライフォーローズ,1,18.10.30,良,13頭,大井ナ,右1600,3番,Ｃ１五 六,5人,482,NaN,笹川翼,56.0,1:44.1,1-1-1-1,39.7,0.1,ロジキャロル,2,18.10.11,良,13頭,大井ナ,右1600,10番,ｎｅｔｋｅｉｂａウマＣ１八 九,9人,484,NaN,赤岡修,56.0,1:45.2,2-2-2-2,42.1,1.6,サムライテイオー,9,18.09.26,不良,11頭,大井ナ,右1600,4番,Ｃ１十一 十二,9人,485,NaN,西啓太,55.0,1:45.4,6-7-8-7,41.0,1.4,マラキア,3-2-1-13,0-0-0-0,3-2-1-13,2-2-1-10
1,20181225,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,9R,19:35,,,サンタクロース賞Ｃ１一 二 三,右1400m,晴,稍重,12,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,2,2,サウンディングベル,牡4,黒鹿毛,スクリーンヒーロー,メダーリア,（アフリート）,藤本現（大井）,56.0,0-0-0-0,中道啓（大井）,伊達敏明,サンシャイン牧場,14.3,(6人気),459,(+5),NaN,6,18.11.14,稍重,12頭,門別,右1200,3番,冬桜特別Ｂ１－２～Ｂ３－１,6人,454,NaN,五十冬,57.0,1:13.8,6-5,38.0,1.7,フランシスコダイゴ,4,18.11.08,稍重,11頭,門別,右1200,10番,シクラメン特別Ｂ３－２Ｂ４－１,3人,456,NaN,五十冬,57.0,1:14.5,5-4,39.0,0.3,シーズアウーマン,5,18.10.25,重,11頭,門別,右1200,5番,みついし花だより特別Ｂ２－２～,2人,452,NaN,五十冬,57.0,1:13.9,5-6,38.2,0.7,ムカワシシャモ,1,18.10.10,重,8頭,門別,右1200,5番,つぼ八特別２Ｂ４－２Ｃ１－１,1人,452,NaN,五十冬,56.0,1:14.0,3-1,38.7,0.4,リボース,2,18.09.27,良,12頭,門別,右1200,7番,クレオメ特別Ｂ４－２Ｃ１－１,2人,458,NaN,五十冬,56.0,1:15.9,3-2,38.9,0.5,ブルーフォース,2-3-5-15,0-0-1-3,1-3-4-7,0-0-0-0
2,20181225,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,9R,19:35,,,サンタクロース賞Ｃ１一 二 三,右1400m,晴,稍重,12,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,http://www2.keiba.go.jp/KeibaWeb/TodayRaceInfo...,3,3,コーラルクリス,牡4,黒鹿毛,シンボリクリスエス,グランフェッテ,（ブラックホーク）,今野忠（川崎）,56.0,0-1-0-0,久保信（大井）,（株）丸若商事,ハシモトフアーム,12.4,(5人気),532,(+2),NaN,2,18.12.04,良,14頭,大井ナ,右1600,3番,Ｃ１三 四,9人,530,NaN,今野忠,56.0,1:44.3,5-4-3-2,39.9,0.4,ネオダラス,9,18.11.12,重,11頭,大井ナ,右1400,2番,Ｃ１一 二 三,6人,533,NaN,笹川翼,56.0,1:31.1,6-6-4,41.4,2.1,ペネトラーレ,6,18.10.31,良,10頭,大井ナ,右1400,8番,いたばし二輪草記念Ｃ１Ｃ２選抜,5人,533,NaN,的場文,57.0,1:29.6,4-5-4,39.8,0.7,カシノランサム,6,18.10.08,良,14頭,大井ナ,右1400,1番,Ｃ１一 二 三,5人,527,NaN,的場文,56.0,1:28.9,7-6-5,39.4,0.4,ストーミースター,7,18.09.18,不良,11頭,大井ナ,右1700,8番,爽籟賞Ｃ１Ｃ２選抜,4人,526,NaN,笹川翼,57.0,1:51.4,5-5-5-4,41.6,0.7,カランツフレイバー,2-2-3-26,0-0-1-5,2-2-2-21,2-2-2-21
3,20181225,http://www2.keiba.go.jp/KeibaWeb/TodayRace

In [7]:
# ファイル名にいれる現在時刻を取得
now_formatted = "{0:%Y%m%d-%H%M%S}".format(datetime.datetime.now())

# excelファイルを出力
horse_df.to_excel('./' + 'horse_df_' + now_formatted + '.xlsx', sheet_name='new_sheet_name')
# tsvファイルを出力
horse_df.to_csv('./' + 'horse_df_' + now_formatted + '.tsv', sep='\t' )
